## Simple Burrows Wheeler Transformation

In [ ]:
a = input()
words = list(a)
bwt = []

In [ ]:
for i in range(len(words)):
    word = a[-1] + a[:-1]
    new = ''.join(word)
    a = new
    bwt.append(new)
    i += 1
print(bwt)

In [ ]:
sort = sorted(bwt)
print(sort)

In [ ]:
output = []
for i in range(len(words)):
    element = sort[i]
    last = element[-1]
    i = i + 1
    output.append(last)
print("".join(output))

## Suffix Array BWT

In [55]:
def suffix_array(string):
    return(list(sorted(range(len(string)), key=lambda i:string[i:])))

def bwt_from_suffix(string, s_array=None):
    if s_array is None:
        s_array = suffix_array(string)
    return("".join(string[idx - 1] for idx in s_array))


def lf_mapping(bwt, letters=None):
    if letters is None:
        letters = set(bwt)
        
    result = {letter:[0] for letter in letters}
    result[bwt[0]] = [1]
    for letter in bwt[1:]:
        for i, j in result.items():
            j.append(j[-1] + (i == letter))
    return(result)


from collections import Counter

def count_occurences(string, letters=None):
    count = 0
    result = {}
    
    if letters is None:
        letters = set(s)
        
    c = Counter(string)
    
    for letter in sorted(letters):
        result[letter] = count
        count += c[letter]
    return result


def update(begin, end, letter, lf_map, counts, string_length):
    beginning = counts[letter] + lf_map[letter][begin - 1] + 1
    ending = counts[letter] + lf_map[letter][end]
    return(beginning,ending)



def generate_all(input_string, s_array=None, eos="$"):
    letters = set(input_string)
    try:
        assert eos not in letters
    
        counts = count_occurences(input_string, letters)

        input_string = "".join([input_string, eos])
        if s_array is None:
            s_array = suffix_array(input_string)
        bwt = bwt_from_suffix(input_string, s_array)
        lf_map = lf_mapping(bwt, letters | set([eos]))

        for i, j in lf_map.items():
            j.extend([j[-1], 0]) # for when pointers go off the edges

        return letters, bwt, lf_map, counts, s_array

    except:
        print("End of string character found in text, deleted EOS from input string")
        input_string = input_string.replace(eos, "")
        letters = set(input_string)
        counts = count_occurences(input_string, letters)

        input_string = "".join([input_string, eos])
        if s_array is None:
            s_array = suffix_array(input_string)
        bwt = bwt_from_suffix(input_string, s_array)
        lf_map = lf_mapping(bwt, letters | set([eos]))

        for i, j in lf_map.items():
            j.extend([j[-1], 0]) # for when pointers go off the edges

        return letters, bwt, lf_map, counts, s_array

    
def find(search_string, input_string, mismatches=0, bwt_data=None, s_array=None):
    
    results = []
     
    if len(search_string) == 0:
        return("Empty Query String")
    if bwt_data is None:
        bwt_data = generate_all(input_string, s_array=s_array)
    
    letters, bwt, lf_map, count, s_array = bwt_data
    
    if len(letters) == 0:
        return("Empty Search String")

    if not set(search_string) <= letters:
        return []

    length = len(bwt)
    

    class Fuzzy(object):
        def __init__(self, **kwargs):
            self.__dict__.update(kwargs)

    fuz = [Fuzzy(search_string=search_string, begin=0, end=len(bwt) - 1,
                        mismatches=mismatches)]

    while len(fuz) > 0:
        p = fuz.pop()
        search_string = p.search_string[:-1]
        last = p.search_string[-1]
        all_letters = [last] if p.mismatches == 0 else letters
        for letter in all_letters:
            begin, end = update(p.begin, p.end, letter, lf_map, count, length)
            if begin <= end:
                if len(search_string) == 0:
                    results.extend(s_array[begin : end + 1])
                else:
                    miss = p.mismatches
                    if letter != last:
                        miss = max(0, p.mismatches - 1)
                    fuz.append(Fuzzy(search_string=search_string, begin=begin,
                                            end=end, mismatches=miss))
    return sorted(set(results))


In [56]:
EOS = "$"
bwt_from_suffix("apple$")

'e$lppa'

In [57]:
generate_all("apple")
## unique letters 
## BWT
## LF Map
## Character Count in unique letters
## Suffix array numbering

({'a', 'e', 'l', 'p'},
 'e$lppa',
 {'e': [1, 1, 1, 1, 1, 1, 1, 0],
  'p': [0, 0, 0, 1, 2, 2, 2, 0],
  'l': [0, 0, 1, 1, 1, 1, 1, 0],
  '$': [0, 1, 1, 1, 1, 1, 1, 0],
  'a': [0, 0, 0, 0, 0, 1, 1, 0]},
 {'a': 0, 'e': 1, 'l': 2, 'p': 3},
 [5, 0, 4, 3, 2, 1])

In [67]:
find("ape", 'apple', mismatches=1)

[0]

## Alice in Wonderland

In [59]:
#%%timeit  ## ~2.44 s ± 377
# Alice in Wonderland Starting from Chapter 8 
import urllib3
url = 'https://www.gutenberg.org/files/11/11-0.txt'
http = urllib3.PoolManager()
text = http.urlopen("GET", url).data.decode()
chapters = text.split("THE END")[0].split("CHAPTER VIII")[2]


## 61235 Characters | 27432 Words | 3762 Lines
#characters = len(text)
#words = len(text.split(" "))
#lines = len(text.split("\n"))



bwt_data = generate_all(chapters)

## 3 instances of "Off with her head"
print("Number of Exact Matches: "+ str(len(find('Off with her head', chapters, mismatches=0, bwt_data=bwt_data))))

## 0 instances of "off with her head" CASE SENSITIVE
print("Number of Exact Matches: "+ str(len(find('off with her head', chapters, mismatches=0, bwt_data=bwt_data))))

### Mismatches = 2
## 5 instances of "Off with her/his head"
print("Number of Fuzzy Matches: "+ str(len(find('Off with her head', chapters, mismatches=2, bwt_data=bwt_data))))

Number of Exact Matches: 3
Number of Exact Matches: 0
Number of Fuzzy Matches: 5


### String based control (default python string search)

In [62]:
%%timeit 
[n for n in range(len(chapters)) if chapters.find("Off with her head", n) == n]

713 ms ± 39.2 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


### Exact matching time!!!

In [64]:
%%timeit 
find('Off with her head', chapters, mismatches=0, bwt_data=bwt_data)

27.4 µs ± 1.34 µs per loop (mean ± std. dev. of 7 runs, 10000 loops each)


### Fuzzy matching time!! 

In [63]:
%%timeit 
find('Off with her head', chapters, mismatches=2, bwt_data=bwt_data)

8.34 ms ± 440 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)
